<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/Creating_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import glob
import pandas as pd
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'Preprocessing_Multitask_Updated' from 'Data_Handling' (/content/Data_Handling.py)

In [3]:
# הורדת הקובץ מ-GitHub
!wget -O Data_Handling.py "https://raw.githubusercontent.com/orifelszer/CrimeData/main/Data_Handling.py"

# ייבוא הפונקציה מהקובץ
from Data_Handling import Preprocessing

--2025-01-06 12:31:15--  https://raw.githubusercontent.com/orifelszer/CrimeData/main/Data_Handling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5256 (5.1K) [text/plain]
Saving to: ‘Data_Handling.py’

Data_Handling.py    100%[===================>]   5.13K  --.-KB/s    in 0s      

2025-01-06 12:31:15 (38.3 MB/s) - ‘Data_Handling.py’ saved [5256/5256]



Define Paths to Crime Datasets (2019–2024)

In [ ]:
!git clone https://github.com/orifelszer/CrimeData.git

import os
import zipfile

zip_folder = 'CrimeData'
zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]

for zip_file in zip_files:
    zip_path = os.path.join(zip_folder, zip_file)
    extract_path = os.path.join(zip_folder, zip_file.replace('.zip', ''))

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted: {zip_file} -> {extract_path}")

Load Crime Data into Pandas DataFrames

In [ ]:
csv_files = glob.glob("CrimeData/**/*.csv", recursive=True)
dataframes = {f"Crimes_{file[-8:-4]}": pd.read_csv(file) for file in csv_files}

In [ ]:
combined_data = pd.concat(dataframes, axis=0, ignore_index=True)

In [ ]:
# הסרת שורות עם ערך -1 בעמודת StatisticGroupKod
combined_data = combined_data[combined_data['StatisticGroupKod'] != -1]

In [ ]:
y_city = combined_data['Yeshuv']  # מיקום הפשע
y_crime_type = combined_data['StatisticGroup']  # סוג הפשע

In [ ]:
# הסרת עמודות החיזוי מהנתונים
features = combined_data.drop(['StatisticGroup', 'StatisticGroupKod', 'Yeshuv', 'YeshuvKod'], axis=1)

# ✅ חלוקה לסטים: המאפיינים מופרדים מעמודות היעד
X_train, X_test, y_city_train, y_city_test, y_crime_train, y_crime_test = train_test_split(
    features, y_city, y_crime_type, test_size=0.2, random_state=42, stratify=y_crime_type)

In [ ]:
# ✅ שימוש בפונקציה המעודכנת ללמידה רב-משתנית
X_train_cleaned, _, _, train_mappings, scaler = Preprocessing_Multitask_Updated(X_train, fit_scaler=True)
X_test_cleaned, _, _, _, _ = Preprocessing_Multitask_Updated(X_test, train_mappings=train_mappings, scaler=scaler)

In [ ]:
# הגדרת עמודות לשמירה על בסיס סט האימון בלבד
columns_to_keep = [col for col in X_train_cleaned.columns if X_train_cleaned[col].sum() >= 10]

# שמירה על עמודות אימון רלוונטיות בלבד
X_train_cleaned = X_train_cleaned[columns_to_keep]

# התאמת סט המבחן רק לפי מה שנקבע באימון
for col in columns_to_keep:
    if col not in X_test_cleaned.columns:
        X_test_cleaned[col] = 0

# הבטחה ששני הסטים באותו סדר עמודות
X_test_cleaned = X_test_cleaned[X_train_cleaned.columns]

In [ ]:
y_city_train = y_city_train.loc[X_train_cleaned.index]
y_city_test = y_city_test.loc[X_test_cleaned.index]
y_crime_train = y_crime_train.loc[X_train_cleaned.index]
y_crime_test = y_crime_test.loc[X_test_cleaned.index]

In [ ]:
# Reducing memory usage
def optimize_data_types(df):
    for col in df.columns:
        col_type = df[col].dtype

        # אם מדובר במשתנה קטגוריאלי טקסטואלי
        if col_type == 'object':
            df[col] = df[col].astype('category')

        # אם מדובר במשתנה מספרי רציף
        elif col_type == 'float64':
            df[col] = df[col].astype('float32')

        # אם מדובר במשתנה מספרי שלם
        elif col_type == 'int64':
            df[col] = df[col].astype('int32')
    return df

# החלת הפונקציה על סט הנתונים
X_train_cleaned = optimize_data_types(X_train_cleaned)
X_test_cleaned = optimize_data_types(X_test_cleaned)

In [ ]:
# קידוד One-Hot עבור עמודות היעד
y_city_train_encoded = pd.get_dummies(y_city_train, prefix="city")
y_city_test_encoded = pd.get_dummies(y_city_test, prefix="city")

y_crime_train_encoded = pd.get_dummies(y_crime_train, prefix="crime")
y_crime_test_encoded = pd.get_dummies(y_crime_test, prefix="crime")

In [ ]:
# שמירת היעדים המקודדים כקבצי CSV
X_train_cleaned.to_csv('X_train.csv', index=False)
X_test_cleaned.to_csv('X_test.csv', index=False)
y_city_train_encoded.to_csv('y_city_train_encoded.csv', index=False)
y_city_test_encoded.to_csv('y_city_test_encoded.csv', index=False)
y_crime_train_encoded.to_csv('y_crime_train_encoded.csv', index=False)
y_crime_test_encoded.to_csv('y_crime_test_encoded.csv', index=False)